#### As an example we could use the Socrata API to mass populate a MYSQL database from the portal
- we have metadata so we can create DDL
- we have CSV or JSON data so we can import

#### let's walk thru populating a single dataset
first we need some DDL generators

In [1]:
from decouple import config
from csv import reader
from mysql import connector

# generate and execute DROP TABLE for specified tablename  
#
def drop_raw_table(mycursor, tablename):
    ddl = "DROP TABLE IF EXISTS `{}`;".format(tablename)
    if mycursor is not None:
        mycursor.execute(ddl)
    return ddl

# generate and execute CREATE TABLE DDL for specified CSV
#
def create_raw_table(mycursor, tablename, csvfile):
    with open(csvfile, encoding='utf-8') as f: 
        r = reader(f)
        fields = next(r)
        
    ddl = 'CREATE TABLE `{}` (\n'.format(tablename)
    for field in fields:
        # truncate to 64 and lower case for MYSQL
        ddl += "`{}` TEXT NULL,\n".format(field.lower()[:64].strip())
    ddl = ddl[:-2] + ");" # remove last ',\n'
    if mycursor is not None:
        mycursor.execute(ddl)
    return ddl

# generate and execute LOAD DATA DDL for specified csv file
# 
def load_raw_table(mycursor, tablename, csvfile):
    ddl = "LOAD DATA INFILE '{}'\n".format(csvfile) 
    ddl += "REPLACE INTO TABLE `{}`\n".format(tablename)
    ddl += """FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"'\n"""
    ddl += "LINES TERMINATED BY '\\n' \n"
    ddl += "IGNORE 1 LINES; "
    if mycursor is not None:
        mycursor.execute(ddl)
    return ddl

print(drop_raw_table(None, 'jx5n-jk6m') + '\n')
print(create_raw_table(None, 'jx5n-jk6m', './jx5n-jk6m.csv') + '\n')
print(load_raw_table(None, 'jx5n-jk6m', './jx5n-jk6m.csv') + '\n')

DROP TABLE IF EXISTS `jx5n-jk6m`;

CREATE TABLE `jx5n-jk6m` (
`age range` TEXT NULL,
`zip code` TEXT NULL,
`city` TEXT NULL,
`gender` TEXT NULL,
`donors` TEXT NULL,
`non-donors` TEXT NULL,
`id card donors` TEXT NULL,
`id card non-donors` TEXT NULL,
`driver license donors` TEXT NULL,
`driver license non-donors` TEXT NULL,
`id card/driver license donors` TEXT NULL,
`id card/driver license non-donors` TEXT NULL);

LOAD DATA INFILE './jx5n-jk6m.csv'
REPLACE INTO TABLE `jx5n-jk6m`
FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"'
LINES TERMINATED BY '\n' 
IGNORE 1 LINES; 



#### Now we need a database connection

In [2]:
class MyDB:
    def __init__(self):
        self._db_connection  = connector.connect(
            host=config('MYSQL_HOST'),
            port=config('MYSQL_PORT'),
            user=config('MYSQL_USER'),
            password=config('MYSQL_PASSWORD'),
            db='odde')
        self._db_cur = self._db_connection.cursor()

    def query(self, query, params=''):
        return self._db_cur.execute(query, params)

    def __del__(self):
        self._db_connection.close()
        
mydb = MyDB()

#### create and load a CSV into a MYSQL table

In [3]:
def create_and_load(csvfile, tablename):
    mycursor = mydb._db_cur
    print(drop_raw_table(mycursor, tablename) + '\n')
    print(create_raw_table(mycursor, tablename, csvfile) + '\n')
    print(load_raw_table(mycursor, tablename , csvfile) + '\n')

# on windows we need to escape the backslash
# TODO is there a platform-agnostic way to build this
create_and_load(config('MYSQL_UPLOADS') + r'\\' + 'jx5n-jk6m.csv', 'jx5n-jk6m-raw')

DROP TABLE IF EXISTS `jx5n-jk6m-raw`;

CREATE TABLE `jx5n-jk6m-raw` (
`age range` TEXT NULL,
`zip code` TEXT NULL,
`city` TEXT NULL,
`gender` TEXT NULL,
`donors` TEXT NULL,
`non-donors` TEXT NULL,
`id card donors` TEXT NULL,
`id card non-donors` TEXT NULL,
`driver license donors` TEXT NULL,
`driver license non-donors` TEXT NULL,
`id card/driver license donors` TEXT NULL,
`id card/driver license non-donors` TEXT NULL);

LOAD DATA INFILE 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\de\\jx5n-jk6m.csv'
REPLACE INTO TABLE `jx5n-jk6m-raw`
FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"'
LINES TERMINATED BY '\n' 
IGNORE 1 LINES; 



#### query the table we just created/loaded to make sure it worked

In [4]:
mycursor = mydb._db_cur
mycursor.execute("select * from `jx5n-jk6m-raw` limit 5")
myresult = mycursor.fetchall()
for x in myresult:
    print(x)  

('71+', '19810', 'WILM', 'M', '0', '1', '0', '1', '0', '1', '0', '0')
('71+', '19805', 'WILMDE', 'F', '0', '1', '0', '1', '0', '1', '0', '0')
('56-60', '19973', 'BLADES', 'M', '1', '1', '0', '2', '1', '1', '0', '0')
('71+', '19804', 'WILMINGTON 5', 'M', '0', '1', '0', '1', '0', '1', '0', '0')
('18-20', '19808', 'MARSHALLTON', 'M', '1', '1', '0', '2', '1', '1', '0', '0')


#### why is this useful? we can use SQL!

#### aggregations for example - assuming we have loaded other datasets from Socrata
e.g. looking at the state's checkbook

In [5]:
# fetch a few records from the checkbook to see what they look like
mycursor.execute("select * from `5s6n-7hpx-raw` limit 5")
myresult = mycursor.fetchall()
for x in myresult:
    print(x) 

# find totals by payees for 2014, ranked in descending order
print ("\n top payees for 2014\n")
mycursor.execute("select vendor, sum(amount) from `5s6n-7hpx-raw` where `fiscal year`='2014' group by vendor order by 2 desc limit 5")
myresult = mycursor.fetchall()
for x in myresult:
    print(x) 


('2014', '2', 'DEPT OF EDUCATION', 'E REACH ACADEMY FOR GIRLS', '1 800 PACK RAT', 'EQUIPMENT RENTAL', 'GENERAL', '640728', '08/08/2013', '300.88')
('2014', '2', 'DEPT OF EDUCATION', 'E REACH ACADEMY FOR GIRLS', '1 800 PACK RAT', 'EQUIPMENT RENTAL', 'GENERAL', '640728', '08/08/2013', '298.02')
('2014', '6', 'EXECUTIVE', 'MAIL/COURIER SERVICES', '100 ENTERPRISE DRIVE', 'OTHER PROFESSIONAL SERVICE', 'GENERAL', '714962', '12/30/2013', '1361.70')
('2014', '6', 'EXECUTIVE', 'FLEET MANAGEMENT', '100 ENTERPRISE DRIVE', 'OTHER PROFESSIONAL SERVICE', 'SPECIAL', '714962', '12/30/2013', '3252.95')
('2014', '6', 'EXECUTIVE', 'SERVICE & INFO GUIDE(SIG)', '100 ENTERPRISE DRIVE', 'OTHER PROFESSIONAL SERVICE', 'GENERAL', '714962', '12/30/2013', '75.65')

 top payees for 2014

('EMPLOYEE COMPENSATION', 1951705229.170117)
('HP ENTERPRISE SVCS LLC', 1708497148.7099996)
('OTHER EMPLOYMENT COSTS BENEFITS', 971788663.5100665)
('HIGHMARK HEALTH SERVICES', 455946734.41999996)
('NORTHERN TRUST COMPANY', 2996313

#### how about joining related datasets - so much easier in SQL than in Pandas!

https://data.delaware.gov/Licenses-and-Certifications/Disciplinary-Actions-for-Professional-and-Occupati/dz6p-akeq/data  
https://data.delaware.gov/Licenses-and-Certifications/Professional-and-Occupational-Licensing/pjnv-eaih/data  

we can join the occupational licensing dataset to the disciplinary dataset to learn a little 

In [6]:
# create a couple of indexes to speed things up
# mycursor.execute("create index index1 on odde.`dz6p-akeq-raw`(license_no(16))")
# mycursor.execute("create index index1 on odde.`pjnv-eaih-raw`(license_no(16))")

mycursor.execute("""select lic.profession_id, count(*) as licensees,
    sum(exists(select * from odde.`dz6p-akeq-raw` disc where disc.license_no=lic.license_no)) as violations,
    100.0 * sum(exists(select * from odde.`dz6p-akeq-raw` disc where disc.license_no=lic.license_no))/count(*) as rate
    from odde.`pjnv-eaih-raw` lic
    where lic.license_status='Active'
    group by lic.profession_id
    order by rate desc""")

myresult = mycursor.fetchall()
for x in myresult:
    print(x) 

('Adult Entertainment', 3, Decimal('1'), Decimal('33.33333'))
('Pilots', 33, Decimal('3'), Decimal('9.09091'))
('Real Estate Appraisers', 764, Decimal('54'), Decimal('7.06806'))
('Land Surveyors', 362, Decimal('24'), Decimal('6.62983'))
('Architecture', 1906, Decimal('117'), Decimal('6.13851'))
('Home Inspectors', 138, Decimal('8'), Decimal('5.79710'))
('Podiatry', 117, Decimal('3'), Decimal('2.56410'))
('Chiropractic', 312, Decimal('8'), Decimal('2.56410'))
('Funeral Services', 280, Decimal('7'), Decimal('2.50000'))
('Mental Health', 712, Decimal('17'), Decimal('2.38764'))
('Landscape Architecture', 211, Decimal('5'), Decimal('2.36967'))
('Real Estate', 6466, Decimal('153'), Decimal('2.36622'))
('Massage Bodywork', 1078, Decimal('25'), Decimal('2.31911'))
('Nursing Home Administrators', 174, Decimal('4'), Decimal('2.29885'))
('Cosmetology and Barbering', 7279, Decimal('162'), Decimal('2.22558'))
('Occupational Therapy', 885, Decimal('18'), Decimal('2.03390'))
('Psychology', 599, Decim